#4

This notebook fine-tunes a pretrained RoBERTa model to detect misogynistic language in online conversations, evaluates it using classification metrics, and provides explainability analysis using SHAP.

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.0 MB/s eta 0:00:00


## Save Fine-Tuned Model

In [3]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import shap
import torch.nn as nn
import torch.nn.functional as F
import json

#############################
# Data Preparation
#############################

# Load dataset
df = pd.read_csv("/content/final_labels.csv")

# Check available columns
print("Available columns:", df.columns)

# Ensure required columns are selected
required_columns = ['body', 'level_1', 'level_2', 'split', 'parent_body']
available_columns = [col for col in required_columns if col in df.columns]
df = df[available_columns].dropna()

# Handle missing parent_body column (if any)
if 'parent_body' not in df.columns:
    df['parent_body'] = "[No Context]"
df['context'] = df['parent_body'] + " [SEP] " + df['body']

# Encode labels for hierarchical classification
level_1_mapping = {label: idx for idx, label in enumerate(df['level_1'].unique())}
df['level_1_label'] = df['level_1'].map(level_1_mapping).astype(int)

level_2_mapping = {label: idx for idx, label in enumerate(df['level_2'].unique())}
df['level_2_label'] = df['level_2'].map(level_2_mapping).astype(int)

# Split dataset (simple train-test split)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

#############################
# Dataset Definition
#############################

class MisogynyDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels_1, labels_2, tokenizer, max_length=256):
        self.texts = texts.tolist()
        self.labels_1 = labels_1.tolist()
        self.labels_2 = labels_2.tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels_1": torch.tensor(self.labels_1[idx], dtype=torch.long),
            "labels_2": torch.tensor(self.labels_2[idx], dtype=torch.long),
        }

#############################
# Model Definition
#############################

# Use a more powerful model for better accuracy
model_name = "roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)

class HierarchicalClassifier(nn.Module):
    def __init__(self, model_name, num_labels_1, num_labels_2):
        super(HierarchicalClassifier, self).__init__()
        self.base_model = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier_1 = nn.Linear(self.base_model.config.hidden_size, num_labels_1)
        self.classifier_2 = nn.Linear(self.base_model.config.hidden_size, num_labels_2)

    def forward(self, input_ids, attention_mask, labels_1=None, labels_2=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # Use [CLS] token representation
        pooled_output = self.dropout(pooled_output)
        logits_1 = self.classifier_1(pooled_output)
        logits_2 = self.classifier_2(pooled_output)

        loss = None
        if labels_1 is not None and labels_2 is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss_1 = loss_fct(logits_1, labels_1)
            loss_2 = loss_fct(logits_2, labels_2)
            loss = loss_1 + loss_2

        # Return a tuple: (loss, logits_1, logits_2)
        return (loss, logits_1, logits_2)

    def save_pretrained(self, save_directory):
        self.base_model.save_pretrained(save_directory + "/base_model")
        torch.save(self.classifier_1.state_dict(), save_directory + "/classifier_1.bin")
        torch.save(self.classifier_2.state_dict(), save_directory + "/classifier_2.bin")
        config = {
            "num_labels_1": self.classifier_1.out_features,
            "num_labels_2": self.classifier_2.out_features,
            "base_model_name": model_name,
            "level_1_mapping": level_1_mapping,
            "level_2_mapping": level_2_mapping
        }
        with open(save_directory + "/config.json", "w") as f:
            json.dump(config, f)

model = HierarchicalClassifier(model_name, len(level_1_mapping), len(level_2_mapping))

#############################
# Create Datasets
#############################

train_dataset = MisogynyDataset(train_df["context"], train_df["level_1_label"], train_df["level_2_label"], tokenizer)
test_dataset = MisogynyDataset(test_df["context"], test_df["level_1_label"], test_df["level_2_label"], tokenizer)

#############################
# Evaluation Metrics
#############################

# Compute overall metrics: a sample is correct only if both predictions are correct.
def compute_metrics(pred):
    labels_1 = pred.label_ids[0]
    labels_2 = pred.label_ids[1]
    logits_1, logits_2 = pred.predictions
    preds_1 = np.argmax(logits_1, axis=-1)
    preds_2 = np.argmax(logits_2, axis=-1)

    overall_accuracy = np.mean((preds_1 == labels_1) & (preds_2 == labels_2))

    precision_1 = precision_score(labels_1, preds_1, average='weighted')
    precision_2 = precision_score(labels_2, preds_2, average='weighted')
    avg_precision = (precision_1 + precision_2) / 2

    recall_1 = recall_score(labels_1, preds_1, average='weighted')
    recall_2 = recall_score(labels_2, preds_2, average='weighted')
    avg_recall = (recall_1 + recall_2) / 2

    f1_1 = f1_score(labels_1, preds_1, average='weighted')
    f1_2 = f1_score(labels_2, preds_2, average='weighted')
    avg_f1 = (f1_1 + f1_2) / 2

    return {
       "overall_accuracy": overall_accuracy,
       "avg_precision": avg_precision,
       "avg_recall": avg_recall,
       "avg_f1": avg_f1
    }

#############################
# Training Setup
#############################

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="overall_accuracy",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train model
trainer.train()

# Save model and tokenizer
model.save_pretrained("./misogyny_model")
tokenizer.save_pretrained("./misogyny_model")
print("Model training complete. Saved to ./misogyny_model")

#############################
# Explainability & Fairness Metrics
#############################

# SHAP Explanation Function using a text masker
def explain_prediction(text, model, tokenizer):
    # Ensure input is a list of strings
    if isinstance(text, str):
        text = [text]
    masker = shap.maskers.Text(tokenizer)

    # Define a wrapper that returns logits from classifier_1
    def model_forward(x):
        # Ensure x is a list of strings
        if isinstance(x, np.ndarray):
            x = x.tolist()
        if not isinstance(x, list):
            x = [str(x)]
        x = [str(s) for s in x]
        inputs = tokenizer(x, return_tensors="pt", truncation=True, padding=True)
        # Move inputs to same device as model
        device = next(model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        _, logits_1, _ = model(**inputs)
        return logits_1.detach().cpu().numpy()

    explainer = shap.Explainer(model_forward, masker)
    shap_values = explainer(text)
    return shap_values

# Compute explainability metric (mean absolute SHAP value)
def compute_explainability_metrics(text, model, tokenizer):
    shap_values = explain_prediction(text, model, tokenizer)
    mean_abs_shap = np.mean(np.abs(shap_values.values))
    return {"mean_absolute_shap": mean_abs_shap}

# Moderation Suggestion Function with positive alternative suggestion
def suggest_moderation(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    _, logits_1, _ = model(**inputs)
    prediction = torch.argmax(logits_1, dim=1).item()

    suggestions = {
        0: "This comment is neutral.",
        1: "Your comment could be more positive. Consider saying: 'I appreciate diverse perspectives and believe we can improve together.'",
        2: "Your comment appears harsh. A more constructive alternative might be: 'I value different opinions and think there is room for improvement in our approach.'"
    }
    return suggestions.get(prediction, "Unable to classify.")

#############################
# Example Usage
#############################

toxic_example_text = "I hate everything about this. It's all terrible."
print("Explanation:", explain_prediction(toxic_example_text, model, tokenizer))
print("Explainability Metrics:", compute_explainability_metrics(toxic_example_text, model, tokenizer))
print("Suggested Alternative:", suggest_moderation(toxic_example_text, model, tokenizer))


Available columns: Index(['entry_id', 'link_id', 'parent_id', 'entry_utc', 'subreddit', 'author',
       'body', 'image', 'label_date', 'week', 'group', 'sheet_order',
       'level_1', 'level_2', 'level_3', 'strength', 'highlight', 'split'],
      dtype='object')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 422264 (422264-national-institute-of-technology-andhrapradesh) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Overall Accuracy,Avg Precision,Avg Recall,Avg F1
1,1.110400,0.837372,0.891686,0.799879,0.894355,0.844481
2,0.928000,1.041850,0.891686,0.799879,0.894355,0.844481
3,1.425100,1.101586,0.891686,0.799879,0.894355,0.844481
4,0.694800,0.995920,0.891686,0.799879,0.894355,0.844481


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

Model training complete. Saved to ./misogyny_model
Explanation: .values =
array([[[ 1.49011612e-08, -7.45058060e-08],
        [ 1.16080046e-04,  1.54644251e-04],
        [ 1.00359321e-04,  1.10983849e-04],
        [ 8.11815262e-05,  9.18358564e-05],
        [ 8.05556774e-05,  9.62615013e-05],
        [ 6.30766153e-05,  7.97510147e-05],
        [ 8.01533461e-05,  1.22427940e-04],
        [ 9.89735126e-05,  1.17346644e-04],
        [ 1.04308128e-04,  1.20431185e-04],
        [ 1.01938844e-04,  1.15633011e-04],
        [ 1.27986073e-04,  1.37463212e-04],
        [ 1.06751919e-04,  1.88440084e-04],
        [ 5.96046448e-08, -2.38418579e-07]]])

.base_values =
array([[ 1.07066035, -1.36709487]])

.data =
(array(['', 'I ', 'hate ', 'everything ', 'about ', 'this', '. ', 'It',
       "'s ", 'all ', 'terrible', '.', ''], dtype=object),)
Explainability Metrics: {'mean_absolute_shap': 9.219119181999794e-05}
Suggested Alternative: This comment is neutral.
